#  Fixed and unfixed particle number and mixed states

In many situations, we are interested in fermionic states with a fixed number of particles. This could happen, for instance, when dealing with the eigenstates of the most common Hamiltonians. Given that situation, we can reduce the basis of our system by a lot. More exactly, if d is the dimension and n the number of particles, we can go from $2^d\times 2^d$ to $\binom{d}{n}\times \binom{d}{n}$ systems. This can be very useful for diagonalizing Hamiltonians, or many other situations.

Another common situation is dealing with states with no fixed particle number. In this case, the best description is given by the $\rho^{\rm qsp}$:


$\begin{equation}
\rho^{\rm qsp} = 
\begin{pmatrix}
\rho^{\rm sp} & \kappa\\
-\kappa^* & I-\rho^{\rm sp}
\end{pmatrix}
\end{equation}$

which has eigenvalues 0 or 1 iff the state is a quasiparticles vacuum. We will learn how to compute this matrix.

Finally, we are going to cover a general mixed state, that is to say a convex combination of pure fermionic states. We will find the resulting one body matrix.

1. [Operators with fixed number](#opfix)
2. [States with fixed number](#statefix)
3. [States with non fixed number](#nonfix)
4. [Mixed states](#mixed)

In [1]:
using Fermionic

<a id="opfix"></a>
## Operators with fixed number


When working with fixed particle number, we are only interested in operators conserving the fermion number, i.e. $c_i^\dagger c_j$, $c_i c_j^\dagger$ and their products. We can access these operators in the following way:

In [2]:
# d = dimension
d = 4
# n = number of fixed particles
n = 2
# i is the mode to be created
i = 2
# j is the mode to be destroyed
j = 4
# this is the c_i^dagger c_j operator
cdc(d,n,i,j)

6×6 SparseArrays.SparseMatrixCSC{Float64,Int64} with 2 stored entries:
  [3, 1]  =  -1.0
  [6, 4]  =  1.0

These operators live in the subspace with fixed particle number, which scales as $\binom{d}{n}$ instead of the infamous $2^d$, with $d$ the dimension and $n$ the number of fixed particles. This basis can be explicitly obtained 

In [3]:
b, index = basis_m(d,n);
Matrix(b)

6×4 Array{Float64,2}:
 0.0  0.0  1.0  1.0
 0.0  1.0  0.0  1.0
 0.0  1.0  1.0  0.0
 1.0  0.0  0.0  1.0
 1.0  0.0  1.0  0.0
 1.0  1.0  0.0  0.0

So the operator connects:

The second output form basis_m(), which is called index, are the positions of each element from the basis b in the whole $2^n$ basis. It can be useful for mapping between basis.

In [4]:
println(Matrix(b)[3,:])
println(Matrix(b)[1,:])
println("--------------------")
println(Matrix(b)[6,:])
println(Matrix(b)[4,:])

[0.0, 1.0, 1.0, 0.0]
[0.0, 0.0, 1.0, 1.0]
--------------------
[1.0, 1.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 1.0]


You can also compute the complex conjugate ccd().
In its core, the funcion cdc() computes the basis of this space. If we wish to compute a combination of these operators, we can directly input the basis to avoid innecesary definitions:

In [5]:
b, index = basis_m(d,n);
cdc(b, index, i, j)

6×6 SparseArrays.SparseMatrixCSC{Float64,Int64} with 2 stored entries:
  [3, 1]  =  -1.0
  [6, 4]  =  1.0

In [6]:
cdc(b, index, i, j)*ccd(b, index, i, j)

6×6 SparseArrays.SparseMatrixCSC{Float64,Int64} with 2 stored entries:
  [3, 3]  =  1.0
  [6, 6]  =  1.0

#### Op_fixed()

As we did with the operators, it is possible to initialize all the possible one body operators through the type Op_fixed.
This is useful when working with Hamiltonians which iterate many times over the same operators (think of two body operators summing over all modes). In order to initalize these operators, we input the dimension and the number of particles.

In [7]:
o = Op_fixed(d,n);

We can then acces the one body operators as follows:

In [8]:
cdc(o, i, j)

6×6 SparseArrays.SparseMatrixCSC{Float64,Int64} with 2 stored entries:
  [3, 1]  =  -1.0
  [6, 4]  =  1.0

We also gain access to several methods, as we did with Op objects. Thw whole list can be found in the index.

### Alternative path
If we have been working in the whole space, but wish to look at the reduced subspace, we will use a function called *fixed()* which takes 2 arguments: the operator we want to reduce and the number of particles we want to fix to. For operators that preserve particle number, this will result in the elimination of empty rows and colums, allowing us to do matrix operations in smaller dimensions.


In [9]:
o = Op(4)
cdcm(o,1,2) 

16×16 SparseArrays.SparseMatrixCSC{Float64,Int64} with 4 stored entries:
  [9 , 5]  =  1.0
  [10, 6]  =  1.0
  [11, 7]  =  1.0
  [12, 8]  =  1.0

Notice we are using a number-preserving operation. We will now fix this operation for 2 particles states.

In [10]:
fixed(cdcm(o,1,2), 2) 

6×6 SparseArrays.SparseMatrixCSC{Float64,Int64} with 2 stored entries:
  [4, 2]  =  1.0
  [5, 3]  =  1.0

Notice the dimension has dropped from 16x16 to 6x6 (in general, from $2^d\times 2^d$ to $\binom{d}{n}\times \binom{d}{n}$). This new state is written in a new reduced basis. We can always access the new basis, with fixed number parity with the function *basis_m()* which also takes two arguments: the operators Op() and the number of particles.

In [11]:
b, _ = basis_m(4,2);
Matrix(b)

6×4 Array{Float64,2}:
 0.0  0.0  1.0  1.0
 0.0  1.0  0.0  1.0
 0.0  1.0  1.0  0.0
 1.0  0.0  0.0  1.0
 1.0  0.0  1.0  0.0
 1.0  1.0  0.0  0.0

So the operator $c_1^\dagger c_2$ connects these two states

In [12]:
println(Matrix(b)[4,:])
println(Matrix(b)[2,:])
println("--------------------")
println(Matrix(b)[5,:])
println(Matrix(b)[3,:])

[1.0, 0.0, 0.0, 1.0]
[0.0, 1.0, 0.0, 1.0]
--------------------
[1.0, 0.0, 1.0, 0.0]
[0.0, 1.0, 1.0, 0.0]


We can do this for every operator, but it only really makes sense for those that are number preseving.
Some really nice examples are the superconducting and the Lipkin hamiltonians, which are also solved in the /examples folder.

<a id="statefix"></a>
## States with fixed number

We already shown how to work with states in order to access some properties, such as the one body matrix and its corresponding entropy. If we are working with fixed number, we can do the same thing. There is a new type now called **State_fixed**.

-  **State_fixed(s::AbstractVector, o::Op, n::Int64)**: initializes a state from a vector **s** (for example an Array{Float64,1}/Array{Complex{Float64},1} or a SparseVector{Float64,Int64}/SparseVector{Complex{Float64},Int64}) with fixed particle number **n**, using the fermionic operators defined by **o**. Once initizialized, functions below are made avaiable. 

These states are written in the basis we obtained with basis_m() and should be normalized for proper results. The arguments for these types are the array/sparse array, the operators and the number of fixed particles.

It is really important that the length matches the length from basis_m(). You can create the state manually as a vector, or use the function *fixed_state()* on a state obtained in the complete basis (for example if you obtained the state through the application of creation operators over the vacuum).


In [13]:
#If we manually create the vector:
stat = zeros(binomial(4,2));  
stat[1] = 1;
stat[6] = 1;
stat = stat/sqrt(stat'*stat)

6-element Array{Float64,1}:
 0.7071067811865475
 0.0
 0.0
 0.0
 0.0
 0.7071067811865475

In [14]:
state = State_fixed(stat,o,2);
rhosp(state)

4×4 SparseArrays.SparseMatrixCSC{Float64,Int64} with 4 stored entries:
  [1, 1]  =  0.5
  [2, 2]  =  0.5
  [3, 3]  =  0.5
  [4, 4]  =  0.5

In [15]:
eigensp(state)

4-element Array{Float64,1}:
 0.5
 0.5
 0.5
 0.5

In [16]:
ssp(state)

1.0

Lets project a vector in the fulled basis to the reduced basis (with fixed particle number) using fixed_state()

In [17]:
using Fermionic
o = Op(4)
s = (cdm(o,1)*cdm(o,2) + cdm(o,3)*cdm(o,4))*vacuum(o);
s = s/sqrt(s'*s)

16-element SparseArrays.SparseVector{Float64,Int64} with 2 stored entries:
  [4 ]  =  0.707107
  [13]  =  0.707107

In [18]:
sf = fixed_state(s, 2) #second argument is the number of particles we want to fix

6-element SparseArrays.SparseVector{Float64,Int64} with 2 stored entries:
  [1]  =  0.707107
  [6]  =  0.707107

In [19]:
state = State_fixed(sf,o,2);
rhosp(state)

4×4 SparseArrays.SparseMatrixCSC{Float64,Int64} with 4 stored entries:
  [1, 1]  =  0.5
  [2, 2]  =  0.5
  [3, 3]  =  0.5
  [4, 4]  =  0.5

We can do the  same for already defined State and output the corresponding State_fixed

In [20]:
o = Op(4)
s = (cdm(o,1)*cdm(o,2) + cdm(o,3)*cdm(o,4))*vacuum(o);
s = s/sqrt(s'*s)
s = State(s,o);

sf = fixed_state(s, 2) #second argument is the number of particles we want to fix
typeof(sf)

State_fixed{SparseArrays.SparseVector{Float64,Int64}}

We can also do the opposite operation, for going from the fixed to the unfixed basis. In order to do that, we use unfixed_state(). It works both for vectors, inputing unfixed_state(s::AbstractVector, n::Int64, num::Int64) with n the dimension and num the number of particles, and for State_fixed, by just inputing the state unfixed_state(s::State_fixed).

In [21]:
o = Op(4)
s = (cdm(o,1)*cdm(o,2) + cdm(o,3)*cdm(o,4))*vacuum(o);
s = s/sqrt(s'*s)
s = State(s,o);

sf = fixed_state(s, 2) #second argument is the number of particles we want to fix
suf = unfixed_state(sf)
typeof(suf)
st(suf) == st(s)

true

<a id="nonfix"></a>

## Non fixed particle number

A general fermionic state has fixed parity but no fixed number. The former formalism can also be used for working with this type of states. We have also defined the $\rho^{\rm qsp}$, which is the matrix defined as followes

$\begin{equation}
\rho^{\rm qsp} = 
\begin{pmatrix}
\rho^{\rm sp} & \kappa\\
-\kappa^* & I-\rho^{\rm sp}
\end{pmatrix}
\end{equation}$

where $\kappa_{ij} = \langle c_j c_i\rangle$ and $-\kappa_{ij}^* = \langle c_j^\dagger c_i^\dagger\rangle$ compose an antisymmetric matrices. The matrix $\rho^{\rm qsp}$ can be diagonalized with Bogoliubov transformations. It is of course invisible to particle-hole transformations. 

In [22]:
op4 = Op(4)
cd1 = cdm(op4,1)
cd2 = cdm(op4,2)
cd3 = cdm(op4,3)
cd4 = cdm(op4,4)
vac = vacuum(op4);

In [23]:
#initialize a state with no fixed fermionic number and real coefficients
state_qsp1 = (cd1*cd2+cd3*cd4+cd1*cd2*cd3*cd4)*vac + vac
state_qsp1 = state_qsp1/sqrt(state_qsp1'*state_qsp1)
state_qsp1 = State(state_qsp1, op4);

In [24]:
Matrix(rhoqsp(state_qsp1))

8×8 Array{Float64,2}:
 0.5   0.0  0.0   0.0   0.0  0.5   0.0  0.0
 0.0   0.5  0.0   0.0  -0.5  0.0   0.0  0.0
 0.0   0.0  0.5   0.0   0.0  0.0   0.0  0.5
 0.0   0.0  0.0   0.5   0.0  0.0  -0.5  0.0
 0.0  -0.5  0.0   0.0   0.5  0.0   0.0  0.0
 0.5   0.0  0.0   0.0   0.0  0.5   0.0  0.0
 0.0   0.0  0.0  -0.5   0.0  0.0   0.5  0.0
 0.0   0.0  0.5   0.0   0.0  0.0   0.0  0.5

In [25]:
#initialize a state with no fixed fermionic number and complex coefficients
state_qsp2 = (cd1*cd2+im*cd1*cd4+cd1*cd2*cd3*cd4)*vac
state_qsp2 = state_qsp2/sqrt(state_qsp2'*state_qsp2)
state_qsp2 = State(state_qsp2, op4);

In [26]:
Matrix(rhoqsp(state_qsp2))

8×8 Array{Complex{Float64},2}:
 1.0+0.0im       0.0+0.0im            0.0+0.0im       …       0.0+0.0im
 0.0+0.0im  0.666667+0.0im            0.0+0.0im               0.0+0.0im
 0.0+0.0im       0.0+0.0im       0.333333+0.0im          0.333333+0.0im
 0.0+0.0im       0.0-0.333333im       0.0+0.0im               0.0+0.0im
 0.0+0.0im       0.0+0.0im            0.0+0.0im               0.0+0.0im
 0.0+0.0im       0.0+0.0im            0.0-0.333333im  …      -0.0-0.333333im
 0.0+0.0im       0.0+0.333333im       0.0+0.0im               0.0+0.0im
 0.0+0.0im       0.0+0.0im       0.333333-0.0im          0.333333+0.0im

It is Hermitian

In [27]:
Matrix(rhoqsp(state_qsp1))' == Matrix(rhoqsp(state_qsp1))

true

We can also check the average number of particles:

In [28]:
n_avg(state_qsp2)

2.666666666666667

<a id="mixed"></a>

## Mixed States

So far, we have only worked with pure states. Nontheless, the most general state is a convex combination of pure states. The one body matrix of a mixed state will be obtained by combining the one body matrices of the pure states involved in the decomposition like so:

$\rho^{\rm sp} = \sum_i p_i \rho_i^{\rm sp}$

We can build this matrix by calling rhosp_mixed(). The argument are two vectors of the same length, the first one containing the probabilities and the second the respective pure states.

In [29]:
p1 = 0.25
p2 = 0.75
p = [p1, p2]

2-element Array{Float64,1}:
 0.25
 0.75

In [30]:
using Fermionic

o = Op(4);

state1 = cdm(o,2)*cdm(o,1)*vacuum(o);
state1 = state1/sqrt(state1'*state1)
state2 = (cdm(o,3)*cdm(o,4)-im*cdm(o,1)*cdm(o,2))*vacuum(o);
state2 = state2/sqrt(state2'*state2);

s1 = State(state1,o); 
s2 = State(state2,o);

s = [s1, s2];

Matrix(rhosp_mixed(p, s))

4×4 Array{Complex{Float64},2}:
 0.625+0.0im    0.0+0.0im    0.0+0.0im    0.0+0.0im
   0.0+0.0im  0.625+0.0im    0.0+0.0im    0.0+0.0im
   0.0+0.0im    0.0+0.0im  0.375+0.0im    0.0+0.0im
   0.0+0.0im    0.0+0.0im    0.0+0.0im  0.375+0.0im

In [31]:
eigensp_mixed(p,s)

4-element Array{Float64,1}:
 0.625
 0.625
 0.375
 0.375

We can also fix the number, as we just learned

In [32]:
using Fermionic

o = Op(4);
nume = 2

p1 = 0.25
p2 = 0.75
p = [p1, p2]

state1 = cdm(o,2)*cdm(o,1)*vacuum(o);
state1 = state1/sqrt(state1'*state1)
state2 = (cdm(o,3)*cdm(o,4)-im*cdm(o,1)*cdm(o,2))*vacuum(o);
state2 = state2/sqrt(state2'*state2);

s1 = fixed_state(state1, nume);
s2 = fixed_state(state2, nume);

s1f = State_fixed(s1, o, nume); #Both states must be the same type!
s2f = State_fixed(s2, o, nume);

s = [s1f, s2f];

Matrix(rhosp_mixed(p, s))

4×4 Array{Complex{Float64},2}:
 0.625+0.0im    0.0+0.0im    0.0+0.0im    0.0+0.0im
   0.0+0.0im  0.625+0.0im    0.0+0.0im    0.0+0.0im
   0.0+0.0im    0.0+0.0im  0.375+0.0im    0.0+0.0im
   0.0+0.0im    0.0+0.0im    0.0+0.0im  0.375+0.0im

In [33]:
eigensp_mixed(p,s)

4-element Array{Float64,1}:
 0.625
 0.625
 0.375
 0.375